# Setup Notebook

In [14]:
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]  # Setting default figure size for plots

# Define Input function (with parameters)

In [15]:
def s_func(t, amplitude=1., shift=0., stretch=1.):
    return amplitude*np.sin(stretch*t)*np.cos(2*stretch*t)+shift

# Plotting and tweaking the output function

In [16]:
def plot_funcs(amplitude=1., shift=0., t_max=1., n_periods=1, resolution=1000):
    t = np.linspace(0, t_max, resolution)  # Create a time array with as many data points as specified by resolution
    stretch = n_periods*2*np.pi/t_max  # Compute stretch parameter to match our number of periods and time frame
    s = s_func(t, amplitude, shift, stretch)
    v = np.diff(s, n=1)  # Get the velocity array (1st derivative of travel)
    a = np.diff(s, n=2) # Get the acceleration array (2nd derivative of travel)
    # Note that by calculating the n-th discrete difference using numpy.diff the resulting array will be n elements shorter than the original array.
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex='col')  # Setup 3 subplots in one column with a common x-axis
    ax3.set_xlabel('time in seconds')
    ax1.set_ylabel('stroke in mm')
    ax2.set_ylabel('velocity in mm/s')
    ax3.set_ylabel('acceleration in mm/s²')
    
    ax1.plot(t, s, color='blue', label='s(t)')
   
    t = t[:-1]  # Remove last element of original time array to match dimension of velocity array
  
    ax2.plot(t, v, color='red', label='v(t)')
    
    t = t[:-1]  # Remove last element of time array to match dimension of acceleration array
  
    ax3.plot(t, a, color='orange', label='a(t)')

    ax1.grid(b=True, which='both')
    ax2.grid(b=True, which='both')
    ax3.grid(b=True, which='both')

    plt.show()
    

In [17]:
interactive(plot_funcs, amplitude=(.001, 100, 1), shift=(0, 10, 1), t_max=(.001, 1, 0.1), n_periods=(1, 10, 1), resolution=fixed(1000))


interactive(children=(FloatSlider(value=1.0, description='amplitude', min=0.001, step=1.0), IntSlider(value=0,…

# Kinematic chain

In [18]:
def r_func(basic_radius, s):
    return basic_radius + s

# Plotting the cam

In [19]:
def plot_cam(basic_radius, t_max, amplitude, shift, n_periods, resolution):
    t = np.linspace(0, t_max, resolution)
    stretch = n_periods*2*np.pi/t_max
    s = s_func(t, amplitude, shift, stretch)
    theta = np.linspace(0, 2*np.pi, resolution)
    r = r_func(basic_radius, s)
    
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.plot(theta, r)
    ax.fill(theta, r, 'b', alpha=0.1)
    ax.grid(True)
    ax.set_title("Cam plot", va='bottom')
    plt.show()
    
    plot_funcs(amplitude, shift, t_max, n_periods, resolution)
    
interactive(plot_cam, basic_radius=(0., 100., 1.), t_max=(0, 3, .1), amplitude=(0, 10, 1), shift=(0, 10, 1), n_periods=(1, 10, 1), resolution=fixed(1000))

interactive(children=(FloatSlider(value=50.0, description='basic_radius', step=1.0), FloatSlider(value=1.0, de…

# Exporting the cam profile

In [20]:
# Finalize cam and function parameters
basic_radius = 50.
t_max = 1.
amplitude = 10.
shift = 0.
n_periods = 2

# Too high a resolution can lead to problems when importing into the CAD application,
# too low a resolution results in a loss of contour accuracy.
resolution = 100

# Get resultant arrays
t = np.linspace(0, t_max, resolution)
stretch = n_periods*2*np.pi/t_max
s = s_func(t, amplitude, shift, stretch)
theta = np.linspace(0, 2*np.pi, resolution)
r = r_func(basic_radius, s)

# Write coordinates to text file
with open('cam_profile.pts', 'w') as f:
    
    # Convert Polar to Cartesian coordinates
    x = r*np.cos(theta)
    y = r*np.sin(theta)
    
    for x, y in zip(x, y):
        z = 0.  #
        coord_str = str(x) + ' ' + str(y) + ' ' + str(z) + '\n'
        f.write(coord_str)